# NTK classifier for Cora

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
from exp_ntk_certify_single import run

In [3]:
def get_str_l(l, precision=2):
    l_str = []
    for el in l:
        l_str.append(f"{el:.{precision}f}")
    return l_str
    

In [4]:
seed = 0

data_params = dict(
    dataset = "cora",
    learning_setting = "inductive", # or "transdructive"
    specification = dict(
        n_per_class = 20,
        fraction_test = 0.1,
        data_dir = "./data",
        make_undirected = True,
        binary_attr = False,
        balance_test = True,
    )
)

model_params = dict(
    label = "GCN",
    model = "GCN",
    normalization = "row_normalization",
    depth = 1,
    #regularizer = 1e-8
    regularizer = 1,
    pred_method = "svm",
)

certificate_params = dict(
    perturbation_model = "l0",
    delta = 5 # l0: local budget = delta * feature_dim
)

verbosity_params = dict(
    debug_lvl = "warning"
)  

other_params = dict(
    device = "0",
    dtype = torch.float64,
    allow_tf32 = False
)

In [5]:
def run_exp(n_seeds, data_params, model_params, certificate_params,
            verbosity_params, other_params):
    acc_l = []
    min_ypred = []
    max_ypred = []
    cond = []
    min_ntklabeled = []
    max_ntklabeled = []
    min_ntkunlabeled = []
    max_ntkunlabeled = []
    for seed in range(n_seeds):
        data_params["specification"]["seed"] = seed
        res = run(data_params, model_params, certificate_params,
                  verbosity_params, other_params, seed)
        acc_l.append(res["accuracy"])
        min_ypred.append(res["min_ypred"])
        max_ypred.append(res["max_ypred"])
        min_ntklabeled.append(res["min_ntklabeled"])
        max_ntklabeled.append(res["max_ntklabeled"])
        min_ntkunlabeled.append(res["min_ntkunlabeled"])
        max_ntkunlabeled.append(res["max_ntkunlabeled"])
        cond.append(res["cond"])
    print(f"Accuracy: {get_str_l(acc_l)}")
    print(f"Min y_pred: {get_str_l(min_ypred)}")
    print(f"Max y_pred: {get_str_l(max_ypred)}")
    print(f"Min NTK_labeled: {get_str_l(min_ntklabeled)}")
    print(f"Max NTK_labeled: {get_str_l(max_ntklabeled)}")
    print(f"Min NTK_unlabeled: {get_str_l(min_ntkunlabeled)}")
    print(f"Max NTK_unlabeled: {get_str_l(max_ntkunlabeled)}")
    print(f"Condition: {get_str_l(cond, precision=0)}")

In [6]:
C = torch.Tensor([[1, 0, 10, 3], [0, 1, 2, 4], [1, 1, 1, 2]])
S = torch.Tensor([[1, 1, 0], [1, 1, 1], [0, 1, 1]])
S_idxadv = S[2,:].view(-1)
SS = S_idxadv.outer(S_idxadv)
print(SS)
cS = torch.einsum("i,jk->ijk",S_idxadv,C)
print(cS[1,2,:])
cST = torch.einsum("ijk->jik",cS)
print(cST[1,2,:])
W = cS + cST + SS.view((SS.shape[0], SS.shape[1], 1))
print(W)
print(W[1,2,:])
W_sort, idx = W.sort(dim=2, descending=True)
print(W_sort)
delta = 2
Wv2 = torch.topk(W,k=delta,dim=2)[0].sum(dim=2)
print(Wv2)
W = W_sort[:,:,:delta].sum(dim=2)
print(W)
cS = torch.einsum("i,jk->ijk",S_idxadv,C[1:3,:])
cST = torch.einsum("i,jk->jik",S_idxadv[1:3],C)
print(cS.shape)
print(cST.shape)
W = cS + cST + SS[:,1:3].view((SS.shape[0], 2, 1))
A = cS
A += cST
A += SS[:,1:3].view((SS.shape[0], 2, 1))
print("check")
print(W)
print(A)
W_sort, idx = W.sort(dim=2, descending=True)
print(W_sort)
delta = 2
W_sort = W_sort[:,:,:delta].sum(dim=2)
print(W_sort)
Wv2 = torch.topk(W,k=delta,dim=2)[0].sum(dim=2)
print(Wv2)
res = torch.zeros((3,3))
res[:,1:3] = Wv2
print(res)
Wv2[Wv2 > 11] = 0
print(Wv2)



tensor([[0., 0., 0.],
        [0., 1., 1.],
        [0., 1., 1.]])
tensor([1., 1., 1., 2.])
tensor([0., 1., 2., 4.])
tensor([[[ 0.,  0.,  0.,  0.],
         [ 1.,  0., 10.,  3.],
         [ 1.,  0., 10.,  3.]],

        [[ 1.,  0., 10.,  3.],
         [ 1.,  3.,  5.,  9.],
         [ 2.,  3.,  4.,  7.]],

        [[ 1.,  0., 10.,  3.],
         [ 2.,  3.,  4.,  7.],
         [ 3.,  3.,  3.,  5.]]])
tensor([2., 3., 4., 7.])
tensor([[[ 0.,  0.,  0.,  0.],
         [10.,  3.,  1.,  0.],
         [10.,  3.,  1.,  0.]],

        [[10.,  3.,  1.,  0.],
         [ 9.,  5.,  3.,  1.],
         [ 7.,  4.,  3.,  2.]],

        [[10.,  3.,  1.,  0.],
         [ 7.,  4.,  3.,  2.],
         [ 5.,  3.,  3.,  3.]]])
tensor([[ 0., 13., 13.],
        [13., 14., 11.],
        [13., 11.,  8.]])
tensor([[ 0., 13., 13.],
        [13., 14., 11.],
        [13., 11.,  8.]])
torch.Size([3, 2, 4])
torch.Size([3, 2, 4])
check
tensor([[[ 1.,  0., 10.,  3.],
         [ 1.,  0., 10.,  3.]],

        [[ 1.,  3.,  5

In [7]:
model_params["regularizer"] = 0.1
model_params["pred_method"] = "svm"
model_params["cache_size"] = 10000
data_params["dataset"] = "cora"
other_params["device"] = 0
certificate_params["delta"] = 1
certificate_params["method"] = "SXXTS"
certificate_params["perturbation_model"] = "l0_del"
verbosity_params["debug_lvl"] = "info"
seed = 0
data_params["specification"]["seed"] = seed
run(data_params, model_params, certificate_params, verbosity_params, other_params, seed)

2024-01-18 14:50:37 (INFO): Starting experiment exp_ntk_certify_single with configuration:
2024-01-18 14:50:37 (INFO): data_params: {'dataset': 'cora', 'learning_setting': 'inductive', 'specification': {'n_per_class': 20, 'fraction_test': 0.1, 'data_dir': './data', 'make_undirected': True, 'binary_attr': False, 'balance_test': True, 'seed': 0}}
2024-01-18 14:50:37 (INFO): model_params: {'label': 'GCN', 'model': 'GCN', 'normalization': 'row_normalization', 'depth': 1, 'regularizer': 0.1, 'pred_method': 'svm', 'cache_size': 10000}
2024-01-18 14:50:37 (INFO): certification_params: {'perturbation_model': 'l0_del', 'delta': 1, 'method': 'SXXTS'}
2024-01-18 14:50:37 (INFO): verbosity_params: {'debug_lvl': 'info'}
2024-01-18 14:50:37 (INFO): other_params: {'device': 0, 'dtype': torch.float64, 'allow_tf32': False}
2024-01-18 14:50:37 (INFO): seed: 0
2024-01-18 14:50:37 (INFO): Currently on gpu device cuda:0
2024-01-18 14:50:37 (INFO): X.min(): 0.0
2024-01-18 14:50:37 (INFO): X.max(): 1.0
2024-

Sig.mean(): 1.0542895379392687
Sig.min(): 0.0
Sig.max(): 27.0
Depth 0
E_der.min(): 0.4999999681690114
E_der.max(): 0.9998576474901678
Sig.mean(): 2.564912753354618
Sig.min(): 0.5152510432128604
Sig.max(): 26.998874259688176
Sig.mean(): 1.0539928175329174
Sig.min(): 0.0
Sig.max(): 21.25
Depth 0
E_der.min(): 0.4999999681690114
E_der.max(): 0.9998576474901678


2024-01-18 14:50:39 (INFO): Accuracy 0.8498168587684631


Sig.mean(): 2.4608704770477776
Sig.min(): 0.5152510432128604
Sig.max(): 16.499312047587217
#Number of Support Vectors
263
alpha_mean: -1.333819710668057e-20
Sig.mean(): 1.0539928175329174
Sig.min(): 0.0
Sig.max(): 21.25
Sig[:,idx_adv]: 0.9606983144631209
Sig_ub.mean(): 1.0539929538978432
Sig_ub.min(): 0.0
Sig_ub.max(): 21.25
Sig_ub[:,idx_adv]: 0.9607906335177738
Sig_lb.mean(): 1.053387521437057
Sig_lb.min(): 0.0
Sig_lb.max(): 21.25
Sig_lb[:,idx_adv]: 0.7537497285901362
Depth 0
tensor(1.0000, device='cuda:0', dtype=torch.float64)
tensor(0., device='cuda:0', dtype=torch.float64)
tensor(1.0000, device='cuda:0', dtype=torch.float64)
tensor(-1.0000e-07, device='cuda:0', dtype=torch.float64)
E_der.mean(): 0.5518514623181219
E_der.min(): 0.4999999681690114
E_der.max(): 0.9998576474901678
E_der[:,idx_adv]: 0.5451175243554843
E_der_lb.mean(): 0.5518234311608536
E_der_lb.min(): 0.4999999681690114
E_der_lb.max(): 0.9998576474901678
E_der_lb[:,idx_adv]: 0.535180199010046
E_der_ub.mean(): 0.5518543

KeyboardInterrupt: 